In [22]:
%reset -f

In [23]:
#!/usr/bin/env python3
import psycopg2
import csv
import glob
import pandas as pd
from psycopg2 import sql
import time
from typing import Union
from datetime import datetime, timezone
import re
import numpy as np
import sys
import math
from tqdm import tqdm
import torch
pd.set_option("display.max_columns", None)

In [24]:
filename = 'Results_1673042557656.txt'

line_number = 1
simulation_parameter_counter = 1
a_spectrum_counter = 1
b_spectrum_counter = 1

lambdas = []
trans = []

thetas = []
pitchs = []
duty_cycles = []
fill_factors =[]
simulations = []
simulation = 1

num_lines = sum(1 for line in open(filename,'r'))
with open(filename, 'r') as file:
    for line in tqdm(file, total=num_lines):
        check1 = line_number == 1 + 4*(simulation_parameter_counter - 1)
        check2 = line_number == 2 + 4*(a_spectrum_counter - 1)
        # check3 = line_number == 3 + 4*(b_spectrum_counter - 1)
        if check1 or check2:
            # Get the grating coupler parameters
            if line_number == 1 + 4*(simulation_parameter_counter - 1):
                split_line = line.split(",")

                theta = float(split_line[0])
                pitch = float(split_line[1])
                duty_cycle = float(split_line[2])
                fill_factor = float(split_line[3])
                
                simulation_parameter_counter += 1

            if line_number == 2 + 4*(a_spectrum_counter - 1):  

                lambda_vals = []
                trans_vals = []

                # Use regular expression to find all words within brackets
                bracketed_words = re.findall(r'\(([^)]+)\)', line)

                # Split the bracketed words into separate words
                vals = [word.split(",") for word in bracketed_words]

                for value in vals:
                    lambda_vals.append(float(value[0])*1000000)
                    trans_vals.append(float(value[1]))
                    lambda_vals.reverse()
                    trans_vals.reverse()

                a_spectrum_counter += 1

            # if line_number == 3 + 4*(b_spectrum_counter - 1):

            #     lambda_vals = []
            #     trans_vals = []

            #     # Use regular expression to find all words within brackets
            #     bracketed_words = re.findall(r'\(([^)]+)\)', line)

            #     # Split the bracketed words into separate words
            #     vals = [word.split(",") for word in bracketed_words]

            #     sorted_vals = sorted(vals, key=lambda x: x[0])

            #     for value in sorted_vals:
            #         lambda_vals.append(float(value[0]))
            #         trans_vals.append(float(value[1]))

            #     b_spectrum_counter += 1

            line_number += 1

            if check2:
                simulations.append(simulation)
                thetas.append(theta)
                pitchs.append(pitch)
                duty_cycles.append(duty_cycle)
                fill_factors.append(fill_factor)
                lambdas.append(lambda_vals)
                trans.append(trans_vals)
                simulation += 1
        else:
            line_number += 1

100%|██████████| 14996/14996 [00:00<00:00, 44331.77it/s]


In [25]:
data = {'simulation':simulations, 'theta':thetas, 'pitch':pitchs, 'duty_cycle':duty_cycles, 'fill_factor':fill_factors, 'lambda':lambdas, 'trans':trans}
dataframe = pd.DataFrame(data=data)
dataframe

,simulation,theta,pitch,duty_cycle,fill_factor,lambda,trans
0,1,10.0000,5.000000e-07,0.400000,0.200000,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000311453, 0.000577727, 0.000898426, 0.0012..."
1,2,10.0000,5.000000e-07,0.400000,0.244444,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[7.77445e-05, 3.47955e-05, 7.99542e-05, 0.0002..."
2,3,10.0000,5.000000e-07,0.400000,0.288889,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000554413, 0.000377262, 0.000212681, 9.5178..."
3,4,10.0000,5.000000e-07,0.400000,0.333333,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000735817, 0.000657348, 0.000511898, 0.0003..."
4,5,10.0000,5.000000e-07,0.400000,0.377778,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000588678, 0.000701883, 0.000749778, 0.0007..."
...,...,...,...,...,...,...,...
3744,3745,13.3333,1.277780e-06,0.577778,0.377778,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000105938, 0.000109693, 0.000117598, 0.0001..."
3745,3746,13.3333,1.277780e-06,0.577778,0.422222,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[5.09078e-05, 4.9765e-05, 4.96777e-05, 5.03599..."
3746,3747,13.3333,1.277780e-06,0.577778,0.466667,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.00020992, 0.000218771, 0.000208879, 0.00018..."
3747,3748,13.3333,1.277780e-06,0.577778,0.511111,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[6.36457e-05, 5.75334e-05, 4.6093e-05, 3.37111..."


In [26]:
dataframe.shape

(3749, 7)

In [27]:
data = {'simulation':[], 'theta':[], 'pitch':[], 'duty_cycle':[], 'fill_factor':[], 'lambda':[], 'trans':[]}
long_dataframe = pd.DataFrame(data=data)

num_rows = dataframe.shape[0]
for i in tqdm(range(num_rows)):
    simulation = dataframe.iloc[i]['simulation']
    theta_val = dataframe.iloc[i]['theta']
    pitch_val = dataframe.iloc[i]['pitch']
    duty_cycle_val = dataframe.iloc[i]['duty_cycle']
    fill_factor_val = dataframe.iloc[i]['fill_factor']

    data = {'simulation':simulation, 'theta':theta_val, 'pitch':pitch_val, 'duty_cycle':duty_cycle_val,
    'fill_factor':fill_factor_val, 'lambda':dataframe.iloc[i]['lambda'], 'trans':dataframe.iloc[i]['trans']}
    new_row = pd.DataFrame(data=data)

    long_dataframe = pd.concat([long_dataframe, new_row])

100%|██████████| 3749/3749 [00:09<00:00, 383.47it/s]


In [28]:
long_dataframe

,simulation,theta,pitch,duty_cycle,fill_factor,lambda,trans
0,1.0,10.0000,5.000000e-07,0.400000,0.200000,1.30000,0.000311
1,1.0,10.0000,5.000000e-07,0.400000,0.200000,1.30308,0.000578
2,1.0,10.0000,5.000000e-07,0.400000,0.200000,1.30618,0.000898
3,1.0,10.0000,5.000000e-07,0.400000,0.200000,1.30929,0.001233
4,1.0,10.0000,5.000000e-07,0.400000,0.200000,1.31241,0.001535
...,...,...,...,...,...,...,...
195,3749.0,13.3333,1.277780e-06,0.577778,0.555556,1.31398,0.000288
196,3749.0,13.3333,1.277780e-06,0.577778,0.555556,1.31085,0.000230
197,3749.0,13.3333,1.277780e-06,0.577778,0.555556,1.30773,0.000165
198,3749.0,13.3333,1.277780e-06,0.577778,0.555556,1.30463,0.000110
